## **My capstone project 1**:
Title: "Cyclistic"  
Author: "HoangHuong"  
Date: "2023-11-20"  

## **Scenario**:
##### **Cyclistic program** :  
* Bike-share program (5,824 bicycle, 692 docking stations)  
* Offering **reclining bikes, hand tricycles, and cargo bikes** (30% use them to commute to work each day)  
* Pricing plan: **single-ride passes,full-day passes,annual memberships**


### **Report**
1. A clear statement of the business task
2. Data sources: <https://divvy-tripdata.s3.amazonaws.com/index.html>
3. Documentation of any cleaning or manipulation of data
4. A summary of your analysis
5. Supporting visualizations and key findings
6. Your top three recommendations based on your analysis

---
**ASK**  
  
*1.What is the problem you are trying to solve?*  
  * Casual riders vs annual members  
  * Why casual riders would buy a membership  
  * How digital media could affecting their marketing tactic(influencing casual riders to become annual membership)  
  * Convert casual riders into annual members  

*2.How can your insights drive business decisions?* 
**Key tasks**
Identify the business task
Consider key stakeholders
**Deliverable**
A clear statement of the business task|



# BUSINESS TASKS
1. Gain insights about Bike-share program. From how casual and anual members obtain some differs to how they may acts differently in kind of benefiting from mambership program. Draw conclusion about how to convert casual riders to annual members and benefits from it.  
2. Stakeholders:
* Founder
* Marketing team
* Business analytics team

# DATA SOURCE
Data sources at: https://divvy-tripdata.s3.amazonaws.com/index.html  
Data was collected over years and quite big. In this scenario, I choose to work with data source in 2023 only.  

**DATASET OVERVIEW**
* Dataset include informations like : dimension variables like **id, start and end coordinates, station name, ridable type...** , measurement varibales : **id count**
* 12 months of data make up : 215 MB ; at least 100.000 rows in each month data ( at most is over 700.000 unique value in June, July and August)  


**DATASET LIMITATION**
* Missing values: about 15% missing value of column **start and end station id, station name**  
* Over years analysis is limitation cause of data range in 1 years.  
* Running big data takes time.
* Database limit to some information, other infor needed for further bussiness analysis such as ride charge, each type of user type fee...

# DATA CLEANING PROCESS
* Uploading dataset into Kaggle for quick overview.
* Join excel files into 1 dataset. 
* Delete rows with all case of missing values.
* Convert data types.
* Create new columns for better analysic: distance, season, duration of ride (day)

# DATA ANALYSIS SUMMARY STATISTICS

**1.Installing and loading packages**

In [ ]:
# find directory path
.libPaths()

In [ ]:
# Specify your custom library path
my_lib_path <- "~/usr/local/lib/R/library"

# Create the directory if it doesn't exist
if (!dir.exists(my_lib_path)) {
  dir.create(my_lib_path, recursive = TRUE)
}

# Set the library path for package installation
.libPaths(my_lib_path)

# Import the packages needed:
packages <- base::list('dplyr', 'ggplot2', 'plyr', 'tidyverse','readxl','openxlsx')

# Function to check and install packages:
pack <- function(pkg) {
  # Identify packages that are not installed
  old_pkg <- packages[!(packages %in% installed.packages(.libPaths()[1], priority = "NA")[, "Package"])]
  
  # Install missing packages
  if (length(old_pkg) > 0) {
    for (i in old_pkg) {
      install.packages(i, lib = my_lib_path, dependencies = TRUE)
      installed_packages <- old_pkg  # Update installed package names
    }
  }
    
  # Attempt to load all packages
  sapply(packages, function(j) {
    if (!require(j, character.only = TRUE, quietly = TRUE)) {
      install.packages(j, lib = my_lib_path)
      library(j, character.only = TRUE)
    }
  })
    
  # Verify packages are installed and loaded
  new_pkg <- packages[(packages %in% installed.packages(.libPaths()[1], priority = "NA")[, "Package"])]
  if (length(new_pkg) == length(packages)) {
    print("All packages are loaded successfully.")
  } else {
    print("Please check pack (not all packages are loaded).")
  }
    return(installed_packages)
}

# Execute the function
pack(packages)
installed_packages <- pack(packages)
print(installed_packages)

**2.Preprocessing dataset**

**a. Combined csv files**

In [ ]:
# Get the list of files in the folder
folder_path <- "/kaggle/input/dataset-1"
csv_files <- list.files(path = folder_path, pattern = "*.csv", full.names = TRUE)[seq(from = 1, to = 12, by = 3)] # can add [1..] to specific range of file


# Initialize an empty list to store data frames
df_list <- list()

# Read each CSV file into a separate data frame and store it in df_list
for (file in csv_files) {
    df <- read.csv(file)
    df_list[[basename(file)]] <- df
}


# combining 12 months data
combined_df <- do.call(rbind, df_list)


# Perform further analysis on combined_df
head(combined_df)
summary(combined_df)


In [ ]:
# Save the combined data frame to a CSV file
output_file_path <- "/kaggle/working/output.csv"
write.csv(combined_df, file = output_file_path, row.names = FALSE)

In [ ]:
# Define the file path (omit previous step)
file_path <- "/kaggle/input/combined/output.csv"

# Read the CSV file
combined_df <- read.csv(file_path)

head(combined_df)


**b. Dealing with multiple types of missing value**

In [ ]:
#replacing empty strings and strings of spaces with NA
library(dplyr)
combined_df <- combined_df %>% mutate_all(~na_if(trimws(.), ""))
summary(combined_df)

# Remove rows with NA values in any column
filtered_df <- na.omit(combined_df)

head(filtered_df)


# View the cleaned data
summary(filtered_df)


**e. Change data type**

In [ ]:
filtered_df$started_at <- as.Date(filtered_df$started_at)
filtered_df$ended_at <- as.Date(filtered_df$ended_at)
filtered_df$start_lng <- as.numeric(as.character(filtered_df$start_lng))
filtered_df$start_lat <- as.numeric(as.character(filtered_df$start_lat))
filtered_df$end_lng <- as.numeric(as.character(filtered_df$end_lng))
filtered_df$end_lat <- as.numeric(as.character(filtered_df$end_lat))

# View the cleaned data
summary(filtered_df)

**d. Add some columns for more detail analysis: distance,season, ride_time(hours)**

In [ ]:
# Function to calculate distance between two points using Haversine formula
haversine_distance <- function(lon1, lat1, lon2, lat2) {
  # Convert latitude and longitude from degrees to radians
  lon1 <- lon1 * pi / 180
  lat1 <- lat1 * pi / 180
  lon2 <- lon2 * pi / 180
  lat2 <- lat2 * pi / 180
  
  # Radius of the Earth in kilometers
  R <- 6371
  
  # Haversine formula
  dlon <- lon2 - lon1
  dlat <- lat2 - lat1
  a <- sin(dlat/2)^2 + cos(lat1) * cos(lat2) * sin(dlon/2)^2
  c <- 2 * atan2(sqrt(a), sqrt(1-a))
  distance <- R * c
  
  return(distance)
}

# Assuming combined_df contains start_longitude, start_latitude, end_longitude, and end_latitude columns

# Calculate distance for each row
filtered_df$distance_km <- haversine_distance(filtered_df$start_lng, filtered_df$start_lat,
                                               filtered_df$end_lng, filtered_df$end_lat)

# Print head of combined_df to verify
head(filtered_df)


In [ ]:
# Function to map months to seasons
get_season <- function(date) {
  month <- lubridate::month(date)
  if (month %in% c(3, 4, 5)) {
    return("Spring")
  } else if (month %in% c(6, 7, 8)) {
    return("Summer")
  } else if (month %in% c(9, 10, 11)) {
    return("Fall")
  } else {
    return("Winter")
  }
}

# Apply the function to create the 'season' column
filtered_df$season <- sapply(filtered_df$started_at, get_season)


# Check the length of the values
head(filtered_df)



In [ ]:
# Ensure datetime format
filtered_df$started_at <- as.POSIXct(filtered_df$started_at, format = "%Y-%m-%d %H:%M:%S", tz = "UTC")
filtered_df$ended_at <- as.POSIXct(filtered_df$ended_at, format = "%Y-%m-%d %H:%M:%S", tz = "UTC")

# Calculate the ride duration in days
filtered_df$ride_duration_days <- difftime(filtered_df$ended_at, filtered_df$started_at, units = "days")

# Preview the updated dataframe to see the new 'ride_time_hours' column
head(filtered_df)

# Filter rides with a duration greater than 0 days
rides_greater_than_zero_days <- filtered_df[filtered_df$ride_duration_days > 0, ]
summary(rides_greater_than_zero_days )

In [ ]:
# Group by day and count distinct ride_ids
day_counts <- filtered_df %>%
  group_by(ride_duration_days) %>%
  summarize(count_distinct_id = n_distinct(ride_id), .groups = 'drop')

# Convert duration to numeric hours
filtered_df <- filtered_df %>%
  mutate(ride_duration_days = as.numeric(ride_duration_days, units = "hours"))

# Plotting the data with enhancements
ggplot(day_counts, aes(x = ride_duration_days, y = count_distinct_id)) +
  geom_col(fill = "steelblue") +  # Using bars to represent the counts
  geom_text(aes(label = count_distinct_id), vjust = -0.3, color = "black") +  # Adding count labels above bars
  #scale_x_discrete(breaks = function(x) seq(min(x), max(x), by = 1)) +  # Adjusting x-axis steps
  theme_minimal() +
  labs(title = "Count of Distinct IDs by day",
       x = "Number of the Day",
       y = "Count of Distinct IDs") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))  # Improve x-axis label readability



Riders mostly commute within a day. There some riders have infomation that ride over 10 days or something, so it may mislead data. So I will filtered them out.

In [ ]:
# Filter out rows with duration greater than 1 day
filtered_df <- filtered_df %>%
  filter(ride_duration_days <= 10)

In [ ]:
# Load necessary libraries
library(dplyr)
library(ggplot2)
library(RColorBrewer)

# Calculate count of unique ride_ids based on member_casual and rideable_type columns
count_df <- filtered_df %>%
  group_by(member_casual, rideable_type) %>%
  summarize(Count = n_distinct(ride_id)) %>%
  ungroup() %>%
  mutate(Total_Count = sum(Count),
         Percentage = (Count / Total_Count) * 100,
         Formatted_Count = prettyNum(Count, big.mark = ",", decimal.mark = "."))

# Create grouped bar chart with stacked bars representing rideable_type within each member_casual group
ggplot(count_df, aes(x = member_casual, y = Count, fill = rideable_type)) +
  geom_bar(stat = "identity") +
  geom_text(aes(label = paste0(Formatted_Count, " (", round(Percentage, 1), "%)")), 
            position = position_stack(vjust = 0.5), color = "black", size = 3) +
  ggtitle("Graph 1: User Type Proportion by Rideable Type") +
  xlab("User Type") +
  ylab("Count of distinct IDs") +
  scale_fill_brewer(palette = "Set3") +
  theme_minimal()


As show above, members take up most of user type. Let see average distance each user type ride.
About 35,1% riders who use classic bike and electric bike still casual riders. This group of user can be targeted to change into members as their choice of bike type.It could be that they can commute more and longer. Let dive more detail.

In [ ]:
install.packages("ggrepel")

In [ ]:
library(ggplot2)
library(dplyr)
library(ggrepel)  # For better text placement

# Calculate total ride counts for each starting location
density_data <- filtered_df %>%
  group_by(start_lat, start_lng, start_station_name) %>%
  summarize(total_rides = n(), .groups = 'drop')  # Using n() to count rides

# Identify top 10 most ridden starting locations
top_10_locations <- density_data %>%
  arrange(desc(total_rides)) %>%  # Make sure to sort by total_rides before selecting top 10
  slice_max(order_by = total_rides, n = 10)

# Print out the top 10 locations
print(top_10_locations)

# Plot the geographic density map with color scale
ggplot() +
  geom_point(data = density_data, aes(x = start_lng, y = start_lat, size = total_rides, color = total_rides), alpha = 0.5) +
  scale_size_continuous(range = c(1, 10)) +  # Adjust point size for better visualization
  scale_color_viridis_c(option = "C") +  # Use a colorblind-friendly palette
  geom_text_repel(
    data = top_10_locations,
    aes(x = start_lng, y = start_lat, label = start_station_name),
    size = 3,
    point.padding = 0.5,  # Increase to reduce the likelihood of text touching the points
    box.padding = 0.5,    # Adjust padding around the text labels
    max.overlaps = 10     # Adjust maximum allowed overlaps
  ) +
  labs(title = "Top 10 Most Ridden Starting Locations", x = "Longitude", y = "Latitude") +
  theme_minimal() +
  theme(legend.position = "right")  # Adjust legend position




Along the seaside seem to be places where most riders took their ride. You can see the density in graph where having most riders. Let see where type of user mostly at.

In [ ]:
library(ggplot2)
library(dplyr)
library(ggrepel)  # For better text placement

# Identify top 10 most ridden starting locations regardless of rider type
top_10_locations <- density_data %>%
  group_by(start_lat, start_lng, start_station_name) %>%
  summarize(total_rides = sum(total_rides), .groups = 'drop') %>%
  arrange(desc(total_rides)) %>%
  slice_max(order_by = total_rides, n = 10)


# Calculate total ride counts for each starting location and rider type
density_data <- filtered_df %>%
  group_by(start_lat, start_lng, start_station_name, member_casual) %>%
  summarize(total_rides = n(), .groups = 'drop')  # Using n() to count rides


# Plot the geographic density map with color scale based on rider type
ggplot() +
  geom_point(data = density_data, aes(x = start_lng, y = start_lat, size = total_rides, color = member_casual), alpha = 0.5) +
  scale_size_continuous(range = c(1, 10)) +  # Adjust point size for better visualization
  scale_color_manual(values = c("casual" = "blue", "member" = "red")) +  # Define custom colors
  geom_text_repel(
    data = top_10_locations,
    aes(x = start_lng, y = start_lat, label = start_station_name),
    size = 3,
    point.padding = 0.5,  # Increase to reduce the likelihood of text touching the points
    box.padding = 0.5,    # Adjust padding around the text labels
    max.overlaps = 10     # Adjust maximum allowed overlaps
  ) +
  labs(title = "Top 10 Most Ridden Starting Locations by Rider Type", x = "Longitude", y = "Latitude") +
  theme_minimal() +
  theme(legend.position = "right")  # Adjust legend position

As graph show, blue represent casual rider, where big blue circle is alsp located in some seaside such as Streeter Dr & Grand Ave. So it means they seem to be potential for member rider. 

All riders tend to active more in Summer and less in Winter.

In [ ]:
# Load necessary libraries
library(dplyr)
library(ggplot2)

# Assuming 'combined_df' is your dataframe and it has columns 'rideable_type' and 'distance_km'
average_distance_by_bike_type <- filtered_df %>%
  group_by(rideable_type) %>%
  summarize(average_distance = mean(distance_km, na.rm = TRUE))

# Print the dataframe to check it
print(average_distance_by_bike_type)



In [ ]:
# Load necessary libraries
library(ggplot2)
library(dplyr)

# Assuming you've already created the average_distance_by_bike_type dataframe
# Plotting the average distance by bike type
ggplot(average_distance_by_bike_type, aes(x = rideable_type, y = average_distance, fill = rideable_type)) +
  geom_col() +  # Using geom_col to create a bar chart
  theme_minimal() +
 geom_text(aes(label = sprintf("%.2f", average_distance)), vjust = -0.3, color = "black") +  # Adding count labels above bars
  labs(title = "Average Distance by Bike Type", x = "Bike Type", y = "Average Distance (km)") +
  scale_fill_brewer(palette = "Set3")  # Optional: using a color palette for aesthetics


In [ ]:
library(ggplot2)
library(dplyr)
library(ggrepel)  # For better text placement

# Calculate total ride counts for each starting location and rideable type
density_data <- filtered_df %>%
  group_by(start_lat, start_lng, start_station_name, rideable_type) %>%
  summarize(total_rides = n(), .groups = 'drop')  # Using n() to count rides

# Identify top 10 most ridden starting locations regardless of rideable type
top_10_locations <- density_data %>%
  group_by(start_lat, start_lng, start_station_name) %>%
  summarize(total_rides = sum(total_rides), .groups = 'drop') %>%
  arrange(desc(total_rides)) %>%
  slice_max(order_by = total_rides, n = 10)

# Plot the geographic density map with color scale based on rideable type
ggplot() +
  geom_point(data = density_data, aes(x = start_lng, y = start_lat, size = total_rides, color = rideable_type), alpha = 0.5) +
  scale_size_continuous(range = c(1, 10)) +  # Adjust point size for better visualization
  scale_color_viridis_d(option = "C") +  # Use a colorblind-friendly palette
  geom_text_repel(
    data = top_10_locations,
    aes(x = start_lng, y = start_lat, label = start_station_name),
    size = 3,
    point.padding = 0.5,  # Increase to reduce the likelihood of text touching the points
    box.padding = 0.5,    # Adjust padding around the text labels
    max.overlaps = 10     # Adjust maximum allowed overlaps
  ) +
  labs(title = "Top 10 Most Ridden Starting Locations by Rideable Type", x = "Longitude", y = "Latitude") +
  theme_minimal() +
  theme(legend.position = "right")  # Adjust legend position


Rider tend to ride along seaside an

In [ ]:
#Draft
# Function to generate the plot based on the specified color variable and color palette
generate_density_plot <- function(color_variable, color_palette) {
  if (identical(color_variable, total_rides)) {
    # If the color variable is total_rides, calculate total rides for each location
    density_data <- filtered_df %>%
      group_by(end_lat, end_lng, end_station_name) %>%
      summarize(total_rides = n(), .groups = 'drop')  # Using n() to count rides
  } else {
    # Otherwise, use the provided color variable directly
    density_data <- filtered_df %>%
      group_by(end_lat, end_lng, end_station_name, {{color_variable}}) %>%
      summarize(total_rides = n(), .groups = 'drop')  # Using n() to count rides
  }
  
  # Identify top 10 most ridden locations regardless of color variable
  top_10_locations <- density_data %>%
    group_by(end_lat, end_lng, end_station_name) %>%
    summarize(total_rides = sum(total_rides), .groups = 'drop') %>%
    arrange(desc(total_rides)) %>%
    slice_max(order_by = total_rides, n = 10)
  
  # Print out the top 10 locations
  print(top_10_locations)
  
  # Plot the geographic density map with color scale based on the specified color variable
  ggplot() +
    geom_point(data = density_data, aes(x = end_lng, y = end_lat, size = total_rides, color = {{color_variable}}), alpha = 0.5) +
    scale_size_continuous(range = c(1, 10)) +  # Adjust point size for better visualization
    geom_text_repel(
      data = top_10_locations,
      aes(x = end_lng, y = end_lat, label = end_station_name),
      size = 3,
      point.padding = 0.5,  # Increase to reduce the likelihood of text touching the points
      box.padding = 0.5,    # Adjust padding around the text labels
      max.overlaps = 30     # Adjust maximum allowed overlaps
    ) +
    scale_color_manual(values = color_palette) +  # Specify custom color palette
    labs(title = paste("Top 10 Most Ridden Ending Locations by", deparse(substitute(color_variable))), x = "Longitude", y = "Latitude") +
    theme_minimal() +
    theme(legend.position = "right")  # Adjust legend position
}

# Define custom color palettes
palette_member_casual <- c("member" = "blue", "casual" = "red")
palette_rideable_type <- c("electric_bike" = "green", "docked_bike" = "blue", "classic_bike" = "orange")

# Call the function with the desired color variables and color palettes
generate_density_plot(member_casual, palette_member_casual)
generate_density_plot(rideable_type, palette_rideable_type)
generate_density_plot(total_rides, palette("viridis"))



# Recommendation
1. Focus to convert from casual to member to those riders along the seaside. Marketing should be focused on those places.
2. Electric bike was most used by riders far from the coast line. So electric bike should be equipted in those area.
3. Riders in area is also potential customers if there are some further analysis and investment in ride environment, equipted and convinence.
